# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper Functions

In [2]:
# Quadratic Weighted Kappa function
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings

def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

## Loading in the Data

In [3]:
train = pd.read_csv('./train.csv')
images = np.load('blindness_images.npy')

In [9]:
images.shape

(3662, 128, 128, 3)

# Train Test Split

In [10]:
X = images
y = train['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

y_train = utils.to_categorical(y_train, 5)
y_test = utils.to_categorical(y_test, 5)

I tried running other models before in a different notebook, however they left me wanting for more approaches that could score better. In my search was Image Augmentation, where the images are manipulated to help the model train on data that wasn't all the same. Doing so should in theory assist the model in identifying underlying patterns. What follows below is my attempt at seeing what stuck.

## 4 Conv Layer Model without Augmentation 

In [18]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
model_test = Sequential()

model_test.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_test.add(MaxPooling2D(2, 2))
model_test.add(Conv2D(64, (3,3), activation='relu'))
model_test.add(MaxPooling2D(2,2))
model_test.add(Conv2D(128, (3,3), activation='relu'))
model_test.add(MaxPooling2D(2,2))
model_test.add(Conv2D(128, (3,3), activation='relu'))
model_test.add(MaxPooling2D(2,2))

model_test.add(Flatten())

model_test.add(Dense(512, activation='relu'))
model_test.add(Dense(5, activation='softmax'))


model_test.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_test = model_test.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 25s 2s/step - loss: 1.3081 - accuracy: 0.4403 - val_loss: 1.1361 - val_accuracy: 0.6343
Epoch 2/100
11/11 [==============================] - 24s 2s/step - loss: 0.9782 - accuracy: 0.6672 - val_loss: 0.8644 - val_accuracy: 0.6943
Epoch 3/100
11/11 [==============================] - 24s 2s/step - loss: 0.8309 - accuracy: 0.7028 - val_loss: 0.8277 - val_accuracy: 0.6900
Epoch 4/100
11/11 [==============================] - 25s 2s/step - loss: 0.7835 - accuracy: 0.7145 - val_loss: 0.8230 - val_accuracy: 0.7074
Epoch 5/100
11/11 [==============================] - 25s 2s/step - loss: 0.7573 - accuracy: 0.7229 - val_loss: 0.8074 - val_accuracy: 0.7009
Epoch 6/100
11/11 [==============================] - 25s 2s/step - loss: 0.7256 - accuracy: 0.7363 - val_loss: 0.7947 - val_accuracy: 0.7216
Epoch 7/100
11/11 [==============================] - 24s 2s/step - loss: 0.6961 - accuracy: 0.7418 - val_loss: 0.7641 - val_accuracy: 0.7249
Epoch 8/100
1

In [19]:
pred_test = model_test.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_test,axis=1))

0.7367323696136592

I tested multiple convolutional layers and was surprised that even without augmentation it performed well. If there was any augmentation that could be done to improve the kappa score, I would happy.

# 4 Conv Layer with Augmentation Layer

In [20]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
data_augmentation2 = Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(128,128,3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.1, 0.3),
    layers.experimental.preprocessing.RandomContrast(0.2),
  ]
)


model_test_aug = Sequential()

model_test_aug.add(data_augmentation2)

model_test_aug.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_test_aug.add(MaxPooling2D(2, 2))
model_test_aug.add(Conv2D(64, (3,3), activation='relu'))
model_test_aug.add(MaxPooling2D(2,2))
model_test_aug.add(Conv2D(128, (3,3), activation='relu'))
model_test_aug.add(MaxPooling2D(2,2))
model_test_aug.add(Conv2D(128, (3,3), activation='relu'))
model_test_aug.add(MaxPooling2D(2,2))

model_test_aug.add(Flatten())

model_test_aug.add(Dense(512, activation='relu'))
model_test_aug.add(Dense(5, activation='softmax'))


model_test_aug.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_test_aug = model_test_aug.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 36s 3s/step - loss: 1.3420 - accuracy: 0.4388 - val_loss: 1.2763 - val_accuracy: 0.4924
Epoch 2/100
11/11 [==============================] - 34s 3s/step - loss: 1.2506 - accuracy: 0.4931 - val_loss: 1.1737 - val_accuracy: 0.4913
Epoch 3/100
11/11 [==============================] - 34s 3s/step - loss: 1.2061 - accuracy: 0.4945 - val_loss: 1.1328 - val_accuracy: 0.5611
Epoch 4/100
11/11 [==============================] - 33s 3s/step - loss: 1.1503 - accuracy: 0.5393 - val_loss: 1.0874 - val_accuracy: 0.6157
Epoch 5/100
11/11 [==============================] - 32s 3s/step - loss: 1.0584 - accuracy: 0.5852 - val_loss: 0.9571 - val_accuracy: 0.6714
Epoch 6/100
11/11 [==============================] - 33s 3s/step - loss: 0.9874 - accuracy: 0.6391 - val_loss: 0.9108 - val_accuracy: 0.6747
Epoch 7/100
11/11 [==============================] - 34s 3s/step - loss: 0.9331 - accuracy: 0.6595 - val_loss: 0.9023 - val_accuracy: 0.6681
Epoch 8/100
1

In [21]:
pred_aug = model_test_aug.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_aug,axis=1))

0.6909811825012597

This model with augmentation did not score as well as I thought it would. It needed further tuning.

# 4 Conv Layer without Contrast and 0.1 Rotation and Zoom

In [22]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
data_augmentation2 = Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(128,128,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1)
  ]
)


model_test_aug2 = Sequential()

model_test_aug2.add(data_augmentation2)

model_test_aug2.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_test_aug2.add(MaxPooling2D(2, 2))
model_test_aug2.add(Conv2D(64, (3,3), activation='relu'))
model_test_aug2.add(MaxPooling2D(2,2))
model_test_aug2.add(Conv2D(128, (3,3), activation='relu'))
model_test_aug2.add(MaxPooling2D(2,2))
model_test_aug2.add(Conv2D(128, (3,3), activation='relu'))
model_test_aug2.add(MaxPooling2D(2,2))

model_test_aug2.add(Flatten())

model_test_aug2.add(Dense(512, activation='relu'))
model_test_aug2.add(Dense(5, activation='softmax'))


model_test_aug2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_test_aug2 = model_test_aug2.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 29s 3s/step - loss: 1.3304 - accuracy: 0.4399 - val_loss: 1.2023 - val_accuracy: 0.4924
Epoch 2/100
11/11 [==============================] - 28s 3s/step - loss: 1.1892 - accuracy: 0.5018 - val_loss: 1.0411 - val_accuracy: 0.6572
Epoch 3/100
11/11 [==============================] - 28s 3s/step - loss: 1.0244 - accuracy: 0.6318 - val_loss: 0.8745 - val_accuracy: 0.6943
Epoch 4/100
11/11 [==============================] - 28s 3s/step - loss: 0.9184 - accuracy: 0.6817 - val_loss: 0.8808 - val_accuracy: 0.6965
Epoch 5/100
11/11 [==============================] - 30s 3s/step - loss: 0.8522 - accuracy: 0.7007 - val_loss: 0.8423 - val_accuracy: 0.7020
Epoch 6/100
11/11 [==============================] - 29s 3s/step - loss: 0.8086 - accuracy: 0.7134 - val_loss: 0.8227 - val_accuracy: 0.7096
Epoch 7/100
11/11 [==============================] - 28s 3s/step - loss: 0.7838 - accuracy: 0.7189 - val_loss: 0.8225 - val_accuracy: 0.7096
Epoch 8/100
1

In [23]:
pred_aug2 = model_test_aug2.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_aug2,axis=1))

0.7334413962799997

Quite good! Further tuning was required.

# Same Model as above with 0.2 Rotation (Best Augmented Model)

In [26]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
data_aug_rot2 = Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(128,128,3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.1)
  ]
)


model_aug_rot2 = Sequential()

model_aug_rot2.add(data_aug_rot2)

model_aug_rot2.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_aug_rot2.add(MaxPooling2D(2, 2))
model_aug_rot2.add(Conv2D(64, (3,3), activation='relu'))
model_aug_rot2.add(MaxPooling2D(2,2))
model_aug_rot2.add(Conv2D(128, (3,3), activation='relu'))
model_aug_rot2.add(MaxPooling2D(2,2))
model_aug_rot2.add(Conv2D(128, (3,3), activation='relu'))
model_aug_rot2.add(MaxPooling2D(2,2))

model_aug_rot2.add(Flatten())

model_aug_rot2.add(Dense(512, activation='relu'))
model_aug_rot2.add(Dense(5, activation='softmax'))


model_aug_rot2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_aug_rot2 = model_aug_rot2.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 30s 3s/step - loss: 1.3429 - accuracy: 0.4348 - val_loss: 1.2497 - val_accuracy: 0.4924
Epoch 2/100
11/11 [==============================] - 28s 3s/step - loss: 1.2228 - accuracy: 0.4931 - val_loss: 1.1618 - val_accuracy: 0.4924
Epoch 3/100
11/11 [==============================] - 29s 3s/step - loss: 1.1801 - accuracy: 0.4967 - val_loss: 1.1257 - val_accuracy: 0.5142
Epoch 4/100
11/11 [==============================] - 29s 3s/step - loss: 1.1233 - accuracy: 0.5466 - val_loss: 1.0756 - val_accuracy: 0.6234
Epoch 5/100
11/11 [==============================] - 28s 3s/step - loss: 1.0544 - accuracy: 0.5929 - val_loss: 0.9412 - val_accuracy: 0.6812
Epoch 6/100
11/11 [==============================] - 29s 3s/step - loss: 0.9621 - accuracy: 0.6424 - val_loss: 0.9325 - val_accuracy: 0.6550
Epoch 7/100
11/11 [==============================] - 28s 3s/step - loss: 0.8846 - accuracy: 0.6668 - val_loss: 0.9123 - val_accuracy: 0.6845
Epoch 8/100
1

In [31]:
pred_aug2 = model_aug_rot2.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_aug2,axis=1))

0.734742720382928

This was the best model so far with the highest Kappa.

# More Tuning

## Conclusion for This Notebook

Unfortunately from here on out, the scores did not get better. I believe the reason for that is because images that are too augmented cause the model to learn what an augmented retina looks like, and it is unable to bring that learning over to non-augmented images. I've left the following unsuccessful models for those who wanted to see the process.

In [28]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
data_aug_rot5 = Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(128,128,3)),
    layers.experimental.preprocessing.RandomRotation(0.5),
    layers.experimental.preprocessing.RandomZoom(0.1)
  ]
)


model_aug_rot5 = Sequential()

model_aug_rot5.add(data_aug_rot2)

model_aug_rot5.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_aug_rot5.add(MaxPooling2D(2, 2))
model_aug_rot5.add(Conv2D(64, (3,3), activation='relu'))
model_aug_rot5.add(MaxPooling2D(2,2))
model_aug_rot5.add(Conv2D(128, (3,3), activation='relu'))
model_aug_rot5.add(MaxPooling2D(2,2))
model_aug_rot5.add(Conv2D(128, (3,3), activation='relu'))
model_aug_rot5.add(MaxPooling2D(2,2))

model_aug_rot5.add(Flatten())

model_aug_rot5.add(Dense(512, activation='relu'))
model_aug_rot5.add(Dense(5, activation='softmax'))


model_aug_rot5.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_aug_rot5 = model_aug_rot5.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 29s 3s/step - loss: 1.3427 - accuracy: 0.4337 - val_loss: 1.2381 - val_accuracy: 0.4924
Epoch 2/100
11/11 [==============================] - 28s 3s/step - loss: 1.2278 - accuracy: 0.4934 - val_loss: 1.1732 - val_accuracy: 0.4924
Epoch 3/100
11/11 [==============================] - 27s 2s/step - loss: 1.1795 - accuracy: 0.4931 - val_loss: 1.1324 - val_accuracy: 0.5098
Epoch 4/100
11/11 [==============================] - 27s 2s/step - loss: 1.1374 - accuracy: 0.5244 - val_loss: 1.0947 - val_accuracy: 0.5764
Epoch 5/100
11/11 [==============================] - 27s 2s/step - loss: 1.0821 - accuracy: 0.5765 - val_loss: 0.9642 - val_accuracy: 0.6605
Epoch 6/100
11/11 [==============================] - 27s 2s/step - loss: 0.9812 - accuracy: 0.6347 - val_loss: 0.9603 - val_accuracy: 0.6496
Epoch 7/100
11/11 [==============================] - 28s 3s/step - loss: 0.8999 - accuracy: 0.6679 - val_loss: 0.8602 - val_accuracy: 0.6910
Epoch 8/100
1

In [30]:
pred_aug5 = model_aug_rot5.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_aug5,axis=1))

0.7269929076392998

In [32]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
data_rot2_zoom2 = Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(128,128,3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2)
  ]
)


model_rot2_zoom2 = Sequential()

model_rot2_zoom2.add(data_aug_rot2)

model_rot2_zoom2.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_rot2_zoom2.add(MaxPooling2D(2, 2))
model_rot2_zoom2.add(Conv2D(64, (3,3), activation='relu'))
model_rot2_zoom2.add(MaxPooling2D(2,2))
model_rot2_zoom2.add(Conv2D(128, (3,3), activation='relu'))
model_rot2_zoom2.add(MaxPooling2D(2,2))
model_rot2_zoom2.add(Conv2D(128, (3,3), activation='relu'))
model_rot2_zoom2.add(MaxPooling2D(2,2))

model_rot2_zoom2.add(Flatten())

model_rot2_zoom2.add(Dense(512, activation='relu'))
model_rot2_zoom2.add(Dense(5, activation='softmax'))


model_rot2_zoom2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_rot2_zoom2 = model_rot2_zoom2.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 30s 3s/step - loss: 1.3432 - accuracy: 0.4297 - val_loss: 1.2722 - val_accuracy: 0.4924
Epoch 2/100
11/11 [==============================] - 28s 3s/step - loss: 1.2284 - accuracy: 0.4931 - val_loss: 1.1694 - val_accuracy: 0.4924
Epoch 3/100
11/11 [==============================] - 29s 3s/step - loss: 1.1792 - accuracy: 0.5018 - val_loss: 1.1104 - val_accuracy: 0.5371
Epoch 4/100
11/11 [==============================] - 27s 2s/step - loss: 1.1277 - accuracy: 0.5408 - val_loss: 1.0502 - val_accuracy: 0.6168
Epoch 5/100
11/11 [==============================] - 28s 3s/step - loss: 1.0681 - accuracy: 0.5834 - val_loss: 0.9458 - val_accuracy: 0.6758
Epoch 6/100
11/11 [==============================] - 28s 3s/step - loss: 0.9730 - accuracy: 0.6431 - val_loss: 0.8722 - val_accuracy: 0.6954
Epoch 7/100
11/11 [==============================] - 28s 3s/step - loss: 0.8871 - accuracy: 0.6755 - val_loss: 0.8843 - val_accuracy: 0.6921
Epoch 8/100
1

In [35]:
pred_rot2_zoom2 = model_rot2_zoom2.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_rot2_zoom2,axis=1))

0.698853915926805

In [36]:
tf.random.set_seed(42)
# https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%202%20-%20Notebook%20(Cats%20v%20Dogs%20Augmentation).ipynb
data_rot2_only = Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(128,128,3)),
    layers.experimental.preprocessing.RandomRotation(0.2)
  ]
)


model_rot2_only = Sequential()

model_rot2_only.add(data_aug_rot2)

model_rot2_only.add(Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)))
model_rot2_only.add(MaxPooling2D(2, 2))
model_rot2_only.add(Conv2D(64, (3,3), activation='relu'))
model_rot2_only.add(MaxPooling2D(2,2))
model_rot2_only.add(Conv2D(128, (3,3), activation='relu'))
model_rot2_only.add(MaxPooling2D(2,2))
model_rot2_only.add(Conv2D(128, (3,3), activation='relu'))
model_rot2_only.add(MaxPooling2D(2,2))

model_rot2_only.add(Flatten())

model_rot2_only.add(Dense(512, activation='relu'))
model_rot2_only.add(Dense(5, activation='softmax'))


model_rot2_only.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history_rot2_only = model_rot2_only.fit(X_train, y_train, validation_data = (X_test, y_test),
                 batch_size = 256,
                 epochs = 100,
                 verbose = 1,
                  callbacks=[early_stop])

Epoch 1/100
11/11 [==============================] - 28s 2s/step - loss: 1.3332 - accuracy: 0.4410 - val_loss: 1.2107 - val_accuracy: 0.4924
Epoch 2/100
11/11 [==============================] - 28s 3s/step - loss: 1.2129 - accuracy: 0.4931 - val_loss: 1.1499 - val_accuracy: 0.4924
Epoch 3/100
11/11 [==============================] - 28s 3s/step - loss: 1.1643 - accuracy: 0.5058 - val_loss: 1.0823 - val_accuracy: 0.5360
Epoch 4/100
11/11 [==============================] - 28s 3s/step - loss: 1.0822 - accuracy: 0.5845 - val_loss: 0.9748 - val_accuracy: 0.6539
Epoch 5/100
11/11 [==============================] - 27s 2s/step - loss: 0.9917 - accuracy: 0.6289 - val_loss: 0.8937 - val_accuracy: 0.6834
Epoch 6/100
11/11 [==============================] - 28s 3s/step - loss: 0.9180 - accuracy: 0.6664 - val_loss: 0.9057 - val_accuracy: 0.6725
Epoch 7/100
11/11 [==============================] - 28s 3s/step - loss: 0.8641 - accuracy: 0.6766 - val_loss: 0.9012 - val_accuracy: 0.6878
Epoch 8/100
1

In [37]:
pred_rot2_only = model_rot2_only.predict(X_test)
quadratic_weighted_kappa(np.argmax(y_test,axis=1), np.argmax(pred_rot2_only,axis=1))

0.7191814746392702